# Linear Programming 

$\text{min }c^Tx$

$\text{s.t.:}$

$Ax = b$

$x\geq0$

$\text{where:}$

$\mathbf{c} = \begin{bmatrix} -3 \\ -2 \\ -1 \\ -5 \\ 0 \\ 0 \\ 0 \end{bmatrix}$

$\mathbf{A} = 
\begin{bmatrix}
    7 & 3 & 4 & 1 & 1 & 0 & 0 \\
    2 & 1 & 1 & 5 & 0 & 1 & 0 \\
    1 & 4 & 5 & 2 & 0 & 0 & 1 \\
\end{bmatrix}
$

$\mathbf{b} = \begin{bmatrix} 7 \\ 3 \\ 8 \end{bmatrix}$

In [1]:
c = [-3; -2; -1; -5; 0; 0; 0] 

A = [7 3 4 1 1 0 0; 
     2 1 1 5 0 1 0; 
     1 4 5 2 0 0 1 ] 

b = [7; 3; 8]

3-element Array{Int64,1}:
 7
 3
 8

## Searching all Basic Feasible Solutions (BFS)

Brute force solution

In [2]:
using Combinatorics 

# Checks if the current basis implies a feasible solution
function is_non_negative(x::Array{Float64, 1}) 
    return length(x[x .< 0]) == 0
end

function searchBFS(c, A, b)
    m, n = size(A)
    @assert rank(A) == m
    
    obj = Inf # Stores objective
    opt_x = zeros(n) # Stores solution
    
    # Each combination represents the indices of columns in A 
    # which corresponds to the indices of basic variables.
    # For each b_idx:
    #     construct the basis matrix B
    #     the cost vector for basic variables c_B
    #     the current value of basic variables x_B = inv(B) * b
    for b_idx in combinations(1:n, m)
        B = A[:, b_idx]
        c_B = c[b_idx]
        x_B = inv(B) * b
        
        # If x_B is non-negative, i.e. feasible, then its objective function value
        # is compared with the smallest objective function value stored in obj
        if is_non_negative(x_B)
            z = dot(c_B, x_B)
            if z < obj 
                obj = z
                opt_x = zeros(n)
                opt_x[b_idx] = x_B
            end
        end
            
        println("Basis:", b_idx)
        println("\t x_B = ", x_B)
        println("\t non-negative? ", is_non_negative(x_B))
        
        if is_non_negative(x_B)
            println("\t obj = ", dot(c_B, x_B))
        end
    end
    
    return opt_x, obj
end

searchBFS (generic function with 1 method)

In [3]:
opt_x, obj = searchBFS(c, A, b)

Basis:[1, 2, 3]
	 x_B = [0.25, 4.75, -2.25]
	 non-negative? false
Basis:[1, 2, 4]
	 x_B = [0.172414, 1.87931, 0.155172]
	 non-negative? true
	 obj = -5.051724137931034
Basis:[1, 2, 5]
	 x_B = [0.571429, 1.85714, -2.57143]
	 non-negative? false
Basis:[1, 2, 6]
	 x_B = [0.16, 1.96, 0.72]
	 non-negative? true
	 obj = -4.3999999999999995
Basis:[1, 2, 7]
	 x_B = [-2.0, 7.0, -18.0]
	 non-negative? false
Basis:[1, 3, 4]
	 x_B = [0.121622, 1.47297, 0.256757]
	 non-negative? true
	 obj = -3.121621621621621
Basis:[1, 3, 5]
	 x_B = [0.777778, 1.44444, -4.22222]
	 non-negative? false
Basis:[1, 3, 6]
	 x_B = [0.0967742, 1.58065, 1.22581]
	 non-negative? true
	 obj = -1.8709677419354835
Basis:[1, 3, 7]
	 x_B = [5.0, -7.0, 38.0]
	 non-negative? false
Basis:[1, 4, 5]
	 x_B = [34.0, -13.0, -218.0]
	 non-negative? false
Basis:[1, 4, 6]
	 x_B = [0.461538, 3.76923, -16.7692]
	 non-negative? false
Basis:[1, 4, 7]
	 x_B = [0.969697, 0.212121, 6.60606]
	 non-negative? true
	 obj = -3.9696969696969693
Basis:[

([0.172414, 1.87931, 0.0, 0.155172, 0.0, 0.0, 0.0], -5.051724137931034)

## Using MathProgBase

In [8]:
using MathProgBase, Clp

solution = linprog(c, A, '=', b, ClpSolver())

println(fieldnames(solution))
println(solution.sol)
println(solution.status)
println(solution.objval)

Symbol[:status, :objval, :sol, :attrs]
[0.172414, 1.87931, 0.0, 0.155172, 0.0, 0.0, 0.0]
Optimal
-5.051724137931035


## Simplex Method
Custom Simplex Method module

In [5]:
include("simplex_method.jl")

using SimplexMethod

# Returns optimal x and objective function values
simplex_method(c, A, b)

([0.172414, 1.87931, 0.0, 0.155172, 0.0, 0.0, 0.0], -5.051724137931035)